cargo los datos del drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive');

Mounted at /content/drive


In [ ]:
# importamos librerias para el manejo de datos pandas para manejo de datos y numpy operaciones con numeros
import numpy as np
import pandas as pd

#import tools

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [30]:
#cargo los datos
train_1=pd.read_csv('/content/drive/MyDrive/bigdata3/proyecto homicidios/columnas_necesarias.csv')

In [31]:
train_1.head()

,Fecha,Departamento,Dia,HoraEstandarizada,Zona,Modalidad,Movil_Agresor,Movil_Victima,Edad,Sexo
0,08/12/2010 12:00:00 AM,VALLE,Miercoles,1,URBANA,ARMA DE FUEGO,A PIE,A PIE,18,MASCULINO
1,01/01/2010 12:00:00 AM,ANTIOQUIA,Viernes,8,URBANA,ARMA DE FUEGO,A PIE,A PIE,32,MASCULINO
2,08/11/2010 12:00:00 AM,NORTE DE SANTANDER,Lunes,7,URBANA,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,A PIE,48,MASCULINO
3,2010-01-01 00:00:00,CAUCA,Lunes,23,URBANA,ARMA DE FUEGO,A PIE,A PIE,26,MASCULINO
4,2010-01-01 00:00:00,CUNDINAMARCA,Domingo,0,URBANA,ARMA BLANCA,A PIE,A PIE,43,MASCULINO


In [ ]:
#Seleccionamos columnas especificas
#train = train[['Fecha', 'Departamento', 'Dia', 'HoraEstandarizada', 'Modalidad', 'Movil_Agresor', 'Movil_Victima', 'Zona', 'Edad', 'Sexo']]
#train.head()

In [32]:
#para la columna fecha
#convierto la columna Fecha a tipo fecha con to_datetime()
df = pd.to_datetime(train_1['Fecha'])
#cargo el train_1 en un dataframe para agregar la columna mes
train_1 = pd.DataFrame(train_1)
#agrego la columna mes al train_1
train_1['Mes'] = df.dt.month

#ya se obtubo el mes de la fecha y se agregó esa nueva columna al train_1
#creamos categorias a paetir de los valores existentes para el mes
train_1=pd.concat([train_1,pd.get_dummies(train_1['Mes'],prefix='Mes_')],axis=1)
#miro como van quedando los datos


#cambio texto por número en el departamento
train_1['Departamento'].replace(
   to_replace= ['AMAZONAS','ANTIOQUIA','ARAUCA','ATLANTICO','BOLIVAR','BOYACA','CALDAS','CAQUETA',
                'CASANARE','CAUCA','CESAR','CHOCO','CORDOBA','CUNDINAMARCA','GUAINIA','GUAJIRA','GUAVIARE','HUILA',
                'MAGDALENA','META','NARINO','NORTE DE SANTANDER','PUTUMAYO','QUINDIO','RISARALDA','SAN ANDRES','SANTANDER',
                'SUCRE','TOLIMA','VALLE','VAUPES','VICHADA'],
    value= [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31],
    inplace=True
)
#luego de cambiar el texto por numeros en el dia, normalizo (valor - min) / (max - min)
df_norm = (train_1['Departamento'] - train_1['Departamento'].min()) / (train_1['Departamento'].max()-train_1['Departamento'].min())
#ya habia agregado el train_1 a un data frame en fecha ahora agrego la columna dia normalizada al train_1
train_1['departamento_normalizado'] = df_norm

#dummies crea categorias a paetir de los valores existentes para Dia
train_1=pd.concat([train_1,pd.get_dummies(train_1['Dia'],prefix='Dia')],axis=1)

#para la columna HoraEstandarizada que se sacó solo la hora mas rápido con excel
train_1=pd.concat([train_1,pd.get_dummies(train_1['HoraEstandarizada'],prefix='Hora_')],axis=1)

#dummies crea categorias a paetir de los valores existentes para Zona
train_1=pd.concat([train_1,pd.get_dummies(train_1['Zona'],prefix='Zona_')],axis=1)

#hacemos lo mismo para la columna Modalidad
train_1=pd.concat([train_1,pd.get_dummies(train_1['Modalidad'],prefix='Modalidad_')],axis=1)

#hacemos lo mismo para la columna Movil_Agresor
train_1=pd.concat([train_1,pd.get_dummies(train_1['Movil_Agresor'],prefix='Movil_Agresor_')],axis=1)

#hacemos lo mismo para la columna Movil_Victima
train_1=pd.concat([train_1,pd.get_dummies(train_1['Movil_Victima'],prefix='Movil_Victima_')],axis=1)

#normalizo la columna edad (valor - min) / (max - min)
df_norm2 = (train_1['Edad'] - train_1['Edad'].min()) / (train_1['Edad'].max()-train_1['Edad'].min())
#ya habia agregado el train_1 a un data frame en fecha ahora agrego la columna edad normalizada al train_1
train_1['edad_normalizada'] = df_norm2

#hacemos lo mismo para la columna sexo
train_1=pd.concat([train_1,pd.get_dummies(train_1['Sexo'],prefix='Sexo_')],axis=1)

train_1.head()

,Fecha,Departamento,Dia,HoraEstandarizada,Zona,Modalidad,Movil_Agresor,Movil_Victima,Edad,Sexo,Mes,Mes__1,Mes__2,Mes__3,Mes__4,Mes__5,Mes__6,Mes__7,Mes__8,Mes__9,Mes__10,Mes__11,Mes__12,departamento_normalizado,Dia_Domingo,Dia_Jueves,Dia_Lunes,Dia_Martes,Dia_Miercoles,Dia_Sabado,Dia_Viernes,Hora__0,Hora__1,Hora__2,Hora__3,Hora__4,Hora__5,Hora__6,Hora__7,Hora__8,...,Modalidad__MOTO BOMBA,Modalidad__OLLA BOMBA,Modalidad__PAQUETE BOMBA,Modalidad__PERSONA BOMBA,Modalidad__POLVORA(FUEGOS PIROTECNICOS),Modalidad__PRENDAS DE VESTIR,Modalidad__PUNZANTES,Modalidad__QUIMICOS,Modalidad__ROCKET,Modalidad__SIN EMPLEO DE ARMAS,Modalidad__SUSTANCIAS TOXICAS,Modalidad__VENENO,Movil_Agresor__A PIE,Movil_Agresor__BICICLETA,Movil_Agresor__CONDUCTOR BUS,Movil_Agresor__CONDUCTOR MOTOCICLETA,Movil_Agresor__CONDUCTOR TAXI,Movil_Agresor__CONDUCTOR VEHICULO,Movil_Agresor__PASAJERO AERONAVE,Movil_Agresor__PASAJERO BARCO,Movil_Agresor__PASAJERO BUS,Movil_Agresor__PASAJERO MOTOCICLETA,Movil_Agresor__PASAJERO TAXI,Movil_Agresor__PASAJERO VEHICULO,Movil_Agresor__VEHICULO,Movil_Victima__A PIE,Movil_Victima__BICICLETA,Movil_Victima__CONDUCTOR BUS,Movil_Victima__CONDUCTOR MOTOCICLETA,Movil_Victima__CONDUCTOR TAXI,Movil_Victima__CONDUCTOR VEHICULO,Movil_Victima__PASAJERO BARCO,Movil_Victima__PASAJERO BUS,Movil_Victima__PASAJERO MOTOCICLETA,Movil_Victima__PASAJERO TAXI,Movil_Victima__PASAJERO VEHICULO,Movil_Victima__VEHICULO,edad_normalizada,Sexo__FEMENINO,Sexo__MASCULINO
0,08/12/2010 12:00:00 AM,29,Miercoles,1,URBANA,ARMA DE FUEGO,A PIE,A PIE,18,MASCULINO,8,0,0,0,0,0,0,0,1,0,0,0,0,0.935484,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.18,0,1
1,01/01/2010 12:00:00 AM,1,Viernes,8,URBANA,ARMA DE FUEGO,A PIE,A PIE,32,MASCULINO,1,1,0,0,0,0,0,0,0,0,0,0,0,0.032258,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.32,0,1
2,08/11/2010 12:00:00 AM,21,Lunes,7,URBANA,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,A PIE,48,MASCULINO,8,0,0,0,0,0,0,0,1,0,0,0,0,0.677419,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.48,0,1
3,2010-01-01 00:00:00,9,Lunes,23,URBANA,ARMA DE FUEGO,A PIE,A PIE,26,MASCULINO,1,1,0,0,0,0,0,0,0,0,0,0,0,0.290323,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.26,0,1
4,2010-01-01 00:00:00,13,Domingo,0,URBANA,ARMA BLANCA,A PIE,A PIE,43,MASCULINO,1,1,0,0,0,0,0,0,0,0,0,0,0,0.419355,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.43,0,1


In [33]:
#borro las columnas que categorice con drop, el implace me dice que ejecuta todo sobre la misma 
# variable, es equivalente a train_1=train_1
train_1.drop(labels=['Fecha'],axis=1,inplace=True)
#train_1.drop(labels=['Departamento'],axis=1,inplace=True)
train_1.drop(labels=['Dia'],axis=1,inplace=True)
train_1.drop(labels=['HoraEstandarizada'],axis=1,inplace=True)
train_1.drop(labels=['Zona'],axis=1,inplace=True)
train_1.drop(labels=['Modalidad'],axis=1,inplace=True)
train_1.drop(labels=['Movil_Agresor'],axis=1,inplace=True)
train_1.drop(labels=['Movil_Victima'],axis=1,inplace=True)
train_1.drop(labels=['Edad'],axis=1,inplace=True)
train_1.drop(labels=['Sexo'],axis=1,inplace=True)
train_1.drop(labels=['Mes'],axis=1,inplace=True)
train_1.head()


,Departamento,Mes__1,Mes__2,Mes__3,Mes__4,Mes__5,Mes__6,Mes__7,Mes__8,Mes__9,Mes__10,Mes__11,Mes__12,departamento_normalizado,Dia_Domingo,Dia_Jueves,Dia_Lunes,Dia_Martes,Dia_Miercoles,Dia_Sabado,Dia_Viernes,Hora__0,Hora__1,Hora__2,Hora__3,Hora__4,Hora__5,Hora__6,Hora__7,Hora__8,Hora__9,Hora__10,Hora__11,Hora__12,Hora__13,Hora__14,Hora__15,Hora__16,Hora__17,Hora__18,...,Modalidad__MOTO BOMBA,Modalidad__OLLA BOMBA,Modalidad__PAQUETE BOMBA,Modalidad__PERSONA BOMBA,Modalidad__POLVORA(FUEGOS PIROTECNICOS),Modalidad__PRENDAS DE VESTIR,Modalidad__PUNZANTES,Modalidad__QUIMICOS,Modalidad__ROCKET,Modalidad__SIN EMPLEO DE ARMAS,Modalidad__SUSTANCIAS TOXICAS,Modalidad__VENENO,Movil_Agresor__A PIE,Movil_Agresor__BICICLETA,Movil_Agresor__CONDUCTOR BUS,Movil_Agresor__CONDUCTOR MOTOCICLETA,Movil_Agresor__CONDUCTOR TAXI,Movil_Agresor__CONDUCTOR VEHICULO,Movil_Agresor__PASAJERO AERONAVE,Movil_Agresor__PASAJERO BARCO,Movil_Agresor__PASAJERO BUS,Movil_Agresor__PASAJERO MOTOCICLETA,Movil_Agresor__PASAJERO TAXI,Movil_Agresor__PASAJERO VEHICULO,Movil_Agresor__VEHICULO,Movil_Victima__A PIE,Movil_Victima__BICICLETA,Movil_Victima__CONDUCTOR BUS,Movil_Victima__CONDUCTOR MOTOCICLETA,Movil_Victima__CONDUCTOR TAXI,Movil_Victima__CONDUCTOR VEHICULO,Movil_Victima__PASAJERO BARCO,Movil_Victima__PASAJERO BUS,Movil_Victima__PASAJERO MOTOCICLETA,Movil_Victima__PASAJERO TAXI,Movil_Victima__PASAJERO VEHICULO,Movil_Victima__VEHICULO,edad_normalizada,Sexo__FEMENINO,Sexo__MASCULINO
0,29,0,0,0,0,0,0,0,1,0,0,0,0,0.935484,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.18,0,1
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0.032258,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.32,0,1
2,21,0,0,0,0,0,0,0,1,0,0,0,0,0.677419,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.48,0,1
3,9,1,0,0,0,0,0,0,0,0,0,0,0,0.290323,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.26,0,1
4,13,1,0,0,0,0,0,0,0,0,0,0,0,0.419355,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.43,0,1


In [34]:
# Y = mX + b  :: Y = wX + b
y = train_1['departamento_normalizado']
print(y)

0         0.935484
1         0.032258
2         0.677419
3         0.290323
4         0.419355
            ...   
102300    0.935484
102301    0.935484
102302    0.935484
102303    0.935484
102304    0.935484
Name: departamento_normalizado, Length: 102305, dtype: float64


In [35]:
x = train_1
x.drop('departamento_normalizado', axis=1, inplace=True)
print(x)

        Departamento  Mes__1  ...  Sexo__FEMENINO  Sexo__MASCULINO
0                 29       0  ...               0                1
1                  1       1  ...               0                1
2                 21       0  ...               0                1
3                  9       1  ...               0                1
4                 13       1  ...               0                1
...              ...     ...  ...             ...              ...
102300            29       0  ...               0                1
102301            29       0  ...               0                1
102302            29       0  ...               1                0
102303            29       0  ...               0                1
102304            29       0  ...               0                1

[102305 rows x 111 columns]


In [36]:
# Separar los datos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.176)

In [39]:
print('== Algunos regresores para prediccion==')
#print('Regresión logística')
KNreg =KNeighborsRegressor()
KNreg.fit(x_train, y_train)
#y_predict = KNreg.predict(x_val)
print('Precisión KNeighborsRegressor (train)', KNreg.score(x_train, y_train))
print('Precisión KNeighborsRegressor (test)', KNreg.score(x_test, y_test))

== Algunos regresores para prediccion==
Precisión Regresión lineal (train) 0.9997203167271609
Precisión Regresión lineal (test) 0.9996174509415428


In [ ]:
#y_predictknnre= KNreg.predict(x_val)
#import matplotlib.pyplot as plt
#plt.plot(y_predictknnre, label='Predicted')
#plt.plot(y_test, label='Expected')
#plt.title("Dia KNeighborsRegressor")
#plt.legend()
#plt.show()

In [ ]:
#guardo el modelo
#import pickle
#rlfile = open("/content/drive/MyDrive/bigdata3/modelos/modelo_diaKNreg.pickle", 'wb')
#pickle.dump(KNreg,rlfile)
#rlfile.close()

In [40]:
print('== Algunos regresores para prediccion==')
#print('DecisionTreeRegressor')
treeR =LinearRegression()
treeR.fit(x_train, y_train)
#y_predict = KNreg.predict(x_val)
print('Precisión Regresión lineal (train)', treeR.score(x_train, y_train))
print('Precisión Regresión lineal (test)', treeR.score(x_test, y_test))

== Algunos regresores para prediccion==
Precisión Regresión lineal (train) 1.0
Precisión Regresión lineal (test) 1.0


In [ ]:
#guardo el modelo
#import pickle
#rlfile = open("/content/drive/MyDrive/bigdata3/modelos/modelo_diaTreeR.pickle", 'wb')
#pickle.dump(treeR,rlfile)
#rlfile.close()

In [ ]:
#y_predictreeR= treeR.predict(x_val)
#import matplotlib.pyplot as plt
#plt.plot(y_predictreeR, label='Predicted')
#plt.plot(y_test, label='Expected')
#plt.title("Dia DecisionTreeRegressor")
#plt.legend()
#plt.show()